In [ ]:
# !pip install tensorflow

In [ ]:
# !pip uninstall ray
# !pip install -U "<PREFIX PATH>\ray-2.0.0.dev0-cp38-cp38-win_amd64.whl"

In [1567]:
# !python --version

Python 3.8.8


In [1]:
import numpy as np
import random
from tensorflow import keras
import ray
np.seterr(all="ignore")


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
ray.init(num_cpus=2)

RayContext(dashboard_url='', python_version='3.8.8', ray_version='2.0.0.dev0', ray_commit='a34dcfce85deaaa4590b7edd68649af16ac8b571', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': 'tcp://127.0.0.1:62214', 'raylet_socket_name': 'tcp://127.0.0.1:65174', 'webui_url': '', 'session_dir': 'C:\\Users\\Roman\\AppData\\Local\\Temp\\ray\\session_2022-04-23_14-05-35_860726_60840', 'metrics_export_port': 63505, 'gcs_address': '127.0.0.1:63884', 'address': '127.0.0.1:63884', 'node_id': '194daac6e14c1e3f4c94f796ae015e1316eb5670290016685585428c'})

In [16]:
ray.shutdown()

In [4]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_d(x):
    return (np.exp(-x))/((np.exp(-x)+1)**2)

def relu(x):
    return np.maximum(0, x)

def relu_d(z):
    return np.greater(z, 0).astype(int)

# softmax
def softmax(x):
    exp_element=np.exp(x-x.max())
    return exp_element/np.sum(exp_element,axis=0)

# derivative of softmax
def softmax_d(x):
    exp_element=np.exp(x-x.max())
    return exp_element/np.sum(exp_element,axis=0)*(1-exp_element/np.sum(exp_element,axis=0))

In [5]:
sigmoid_activator = (sigmoid, sigmoid_d)
relu_activator = (relu, relu_d)
softmax_activator = (softmax, softmax_d)

In [6]:
def mse_loss(outputs, target):
    return ((outputs - target) ** 2).sum() / len(outputs)

def mse_loss_d(outputs, target):
    return 2 * (outputs - target) / len(outputs)

def cross_entropy(model_output, target):
    if model_output.shape != target.shape:
        raise ValueError('Dimensions of model output and target do not match')
    eps = np.finfo(model_output.dtype).eps
    model_output = np.clip(model_output, eps, 1- eps)
    return -np.sum(target * np.log(model_output))/model_output.shape[0]

def cross_entropy_d(model_output, target):
    return model_output - target

In [7]:
mse_loss_set = (mse_loss, mse_loss_d)
crossentropy_loss_set = (cross_entropy, cross_entropy_d)

In [8]:
def layer(neurons, fset, bias=True):
    act = fset[0] if fset is not None else None
    actd = fset[1] if fset is not None else None
    
    return {
        'activation': act,
        'activation_d': actd,
        'neurons': neurons,
        'bias': bias
    }

In [12]:
class RMSPropOptimizer(object):
    def __init__(self, model, alpha, momentum, forget_factor):
        self.alpha = alpha
        self.momentum = momentum
        self.forget_factor = forget_factor
        self.model = model
        self.deltas = [np.zeros(wmat) for wmat in self.model.weights_layout()]
        self.rms_prop_deltas = [np.zeros(wmat) for wmat in self.model.weights_layout()]

    def step(self, grad):
        eps = 1e-8
        for i, wg in enumerate(grad):
            # RMSProp
            v = self.forget_factor * self.rms_prop_deltas[i] + \
                (1 - self.forget_factor) * (wg ** 2)

            # delta for momentum
            delta = self.alpha * wg / (v ** 0.5 + eps) + \
                    self.momentum * self.deltas[i]

            self.model.weights[i] -= delta
            self.deltas[i] = delta
            self.rms_prop_deltas[i] = v



In [30]:
np.array((4,))

array([4])

In [11]:
import ray

class RayGradientCalculator(object):
    @ray.remote
    class Actor(object):
        # initializer is a dict of arguments to a model copy
        def __init__(self, initializer):
            initializer['initialize'] = False
            self.model = Model(**initializer)
            self.model.reset_inputs()
            self.model.reset_outputs()

        def calc_weight_grad(self, inp, tar, weights):
            self.model.weights = weights
            self.model.forward(inp)
            grad = self.model.calc_input_grad(tar)
            return self.model.calc_weight_grad(grad)

    def __init__(self, model, workers):
        self.max_workers = workers
        self.model = model
        self.initializer = None
        self.pool = None
        self._initialize()


    def _initialize(self):
        self.initializer = {
            'layers': self.model.layers,
            'loss_set': (self.model.loss, self.model.loss_d),
            'dtype': self.model.dtype,
            'initialize': False
        }

        self.pool = []
        for _ in range(self.max_workers):
            self.pool.append(RayGradientCalculator.Actor.remote(self.initializer))

    def _get_actor(self):
        if len(self.pool) == 0:
            return None
        return self.pool.pop()

    def _release_actors(self, *actors):
        for actor in actors:
            if actor not in self.pool:
                self.pool.append(actor)

    def run(self, batch):
        batch_size = len(batch)
        if batch_size == 0:
            return

        object_refs = []

        weights_ref = ray.put(self.model.weights)
        object_refs.append(weights_ref)

        result_w_grad = None

        idx = 0

        while idx < len(batch):
            gradient_tasks = []
            actors = []

            while idx < len(batch):
                trainer = self._get_actor()
                if trainer is None:
                    break

                inp, tar = batch[idx]
                idx += 1

                inp_ref = ray.put(inp)
                tar_ref = ray.put(tar)

                object_refs.append(inp_ref)
                object_refs.append(tar_ref)
                actors.append(trainer)

                task = trainer.calc_weight_grad.remote(inp_ref, tar_ref, weights_ref)
                gradient_tasks.append(task)

            grads = ray.get(gradient_tasks)

            w_grad = list(map(lambda x: sum(x) / batch_size, zip(*grads)))

            if result_w_grad is None:
                result_w_grad = w_grad
            else:
                for i in range(len(w_grad)):
                    result_w_grad[i] += w_grad[i]

            del gradient_tasks
            self._release_actors(*actors)

        del object_refs

        return result_w_grad

In [13]:
class Model(object):

    def __init__(self, 
                 layers, 
                 loss_set,
                 initialize=True,
                 dtype=np.float64,
                 ):
        self.layers = layers
        self.dtype = dtype
        self.loss = loss_set[0]
        self.loss_d = loss_set[1]
        # self.trainers = []

        self.weights = None
        self.inputs = None
        self.outputs = None

        if initialize:
            self.initialize()

    def zero_array(self, shape):
        return np.zeros(shape, dtype=self.dtype)

    def random_array(self, shape):
        return np.random.random(shape).astype(self.dtype)

    def layer(self, layer):
        return self.layers[layer]
    
    def has_bias(self, layer):
        # output layer can't have a bias
        if layer == self.count_layers() - 1:
            return False
        return self.layer(layer)['bias']
    
    # returns an activation function for the layer
    def activator(self, layer):
        return self.layer(layer)['activation']
    
    # returns a derivative of an activation function for the layer
    def activator_d(self, layer):
        return self.layer(layer)['activation_d']
    
    # returns the amount of neurons in the layer
    def count_neurons(self, layer):
        bias = int(self.has_bias(layer))
        return self.layer(layer)['neurons'] + bias
    
    # returns the amount of layers in the model
    def count_layers(self):
        return len(self.layers)
    
    def reset_outputs(self):
        self.outputs = []
        for i in range(self.count_layers()):
            neurons = self.count_neurons(i)
            self.outputs.append(self.zero_array((neurons,)))
    
    def reset_inputs(self):
        self.inputs = []
        for i in range(self.count_layers()):
            neurons = self.count_neurons(i)
            self.inputs.append(self.zero_array((neurons,)))
    
    def reset_weights(self):
        self.weights = []
        for i in range(self.count_layers()):
            neurons = self.count_neurons(i)
            if i > 0:
                w_shape = (self.count_neurons(i - 1), neurons)
                self.weights.append(self.random_array(w_shape))
            
    def initialize(self):
        self.reset_inputs()
        self.reset_outputs()
        self.reset_weights()

    def forward(self, x):
        if len(self.inputs[0]) - int(self.has_bias(0)) != len(x):
            raise Exception('incorrect input size')

        for i, val in enumerate(x):
            self.inputs[0][i] = val
            self.outputs[0][i] = val

        if self.has_bias(0):
            self.inputs[0][-1] = 1
            self.outputs[0][-1] = 1

        for i in range(1, self.count_layers()):
            activate = self.activator(i)
            self.inputs[i] = self.outputs[i-1] @ self.weights[i-1]
            if activate is not None:
                self.outputs[i] = activate(self.inputs[i])
            else:
                self.outputs[i] = self.inputs[i].copy()

            if self.has_bias(i):
                self.outputs[i][-1] = 1
                self.inputs[i][-1] = 1

        return self.outputs[-1].copy()
    
    def calc_input_grad(self, target):
        grad = [
            self.zero_array((self.count_neurons(i),))
                for i in range(self.count_layers())
        ]

        output_layer = self.count_layers() - 1
        o_activate_d = self.activator_d(output_layer)
        o_outputs = self.outputs[output_layer]
        o_inputs = self.inputs[output_layer]
        grad[output_layer] = self.loss_d(o_outputs, target)
        
        if o_activate_d is not None:
            grad[output_layer] *= o_activate_d(o_inputs)            
        for i in range(self.count_layers() - 2, 0, -1):
            activate_d = self.activator_d(i)
            grad[i] = grad[i + 1] @ self.weights[i].T
            if activate_d is not None:
                grad[i] *= activate_d(self.inputs[i])
        return grad
    
    def weights_layout(self):
        return [wmat.shape for wmat in self.weights]

    def calc_weight_grad(self, input_grad):
        layers = self.count_layers()
        w_grad = [None] * (layers - 1)
        for output_layer in range(layers-1):
            inputs = self.outputs[output_layer]
            grad = input_grad[output_layer + 1]
            w_grad[output_layer] = np.outer(inputs, grad)
        return w_grad

test_model = Model(
    (
        layer(3, None, False),
        layer(2, None, False),
        layer(1, None, False)
    ),
    mse_loss_set
)

test_model.weights = [
    np.array([
        [0.1, 0.2],
        [0.3, 0.4],
        [0.5, 0.6]
    ]),
    np.array([
        [0.3],
        [0.6]
    ])
]

# test_model.log_pool_info()

# expected result is 2.34
test_model.forward([1, 2, 3])

array([2.34])

In [22]:
layout = (
    layer(2, None),
    layer(2, sigmoid_activator),
    layer(1, None)
)

xor_model = Model(layout, mse_loss_set)

grad_calc = RayGradientCalculator(xor_model, 4)
optimizer = RMSPropOptimizer(xor_model, 0.1, 0.01, 0.01)

dataset = [
    (np.array([0, 0]), np.array([0])),
    (np.array([0, 1]), np.array([1])),
    (np.array([1, 0]), np.array([1])),
    (np.array([1, 1]), np.array([0])),
]

for inp, tar in dataset:
    print(inp, xor_model.forward(inp))
print()

for epoch in range(4000):
#     print(epoch)
    if epoch % 400 == 0:
        print('epoch:', epoch)
        for inp, tar in dataset:
            print(inp, xor_model.forward(inp))
        # xor_model.log_pool_info()
        print('--------------------')
    grad = grad_calc.run(dataset)
    optimizer.step(grad)

for inp, tar in dataset:
    print(inp, xor_model.forward(inp))

[0 0] [0.73714542]
[0 1] [0.759092]
[1 0] [0.80487922]
[1 1] [0.82386688]

epoch: 0
[0 0] [0.73714542]
[0 1] [0.759092]
[1 0] [0.80487922]
[1 1] [0.82386688]
--------------------
epoch: 400
[0 0] [-0.0993933]
[0 1] [0.66363996]
[1 0] [0.66363091]
[1 1] [0.3613894]
--------------------
epoch: 800
[0 0] [-0.12663424]
[0 1] [0.71768317]
[1 0] [0.71768227]
[1 1] [0.27750187]
--------------------
epoch: 1200
[0 0] [-0.14083222]
[0 1] [0.75160192]
[1 0] [0.75160182]
[1 1] [0.1990651]
--------------------
epoch: 1600
[0 0] [-0.14968919]
[0 1] [0.77705273]
[1 0] [0.77705272]
[1 1] [0.13386574]
--------------------
epoch: 2000
[0 0] [-0.15553041]
[0 1] [0.79656388]
[1 0] [0.79656388]
[1 1] [0.08188127]
--------------------
epoch: 2400
[0 0] [-0.15949746]
[0 1] [0.81149917]
[1 0] [0.81149917]
[1 1] [0.04135619]
--------------------
epoch: 2800
[0 0] [-0.16224065]
[0 1] [0.82286603]
[1 0] [0.82286603]
[1 1] [0.01026989]
--------------------
epoch: 3200
[0 0] [-0.16415667]
[0 1] [0.83146467]
[1 0]

In [24]:
import torchvision
import torch

transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])
torch_ds = torchvision.datasets.MNIST('/files/', train=True, download=True)

In [14]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [29]:
# torch_ds[54][0]

In [14]:
def flatten(t):
    return [item for sublist in t for item in sublist]

train_dataset = []

for i in range(len(torch_ds)):
    img, tar = torch_ds[i]
    inp = transform(img).numpy().flatten()
    target = [0] * 10
    target[tar] = 1
    train_dataset.append((inp, target, img))
    


In [15]:
layout = (
    layer(784, None),
    layer(64, sigmoid_activator),
    layer(10, sigmoid_activator),
)

model = Model(layout, mse_loss_set)

gradient_calculator = RayGradientCalculator(model, 64)
rms_prop_optimizer = RMSPropOptimizer(model, 0.01, 0.01, 0.01)


In [16]:
BATCH_SIZE = 64
batches = []

next_batch = []

for inp, tar, img in train_dataset:
    next_batch.append((inp, tar))
    if len(next_batch) == BATCH_SIZE:
        batches.append(next_batch)
        next_batch = []

In [17]:
def test(model, train_dataset, count=512):
    guessed = 0
    for _ in range(count):
        inp, tar, _ = random.choice(train_dataset)
        res = model.forward(inp)
        if tar[list(res).index(max(res))] == 1:
            guessed += 1
    return guessed / count

In [18]:
model.weights_layout()

[(785, 65), (65, 10)]

In [19]:
from IPython.display import clear_output
import time

err = []
guessed = 0

testing_step = 100

for epoch in range(50):
    print('@@@@@@@@@@@@@@@@@@@@@')
    print(f'EPOCH: {epoch+1}')
    print('@@@@@@@@@@@@@@@@@@@@@')

    last_test_time = time.time()
    for i, batch in enumerate(batches):
        grad = gradient_calculator.run(batch)
        # print(list(map(lambda x: x.shape, grad)))
        rms_prop_optimizer.step(grad)

        if i % testing_step == testing_step - 1:
            now = time.time()
            elapsed = now - last_test_time
            last_test_time = now
            
            time_per_epoch = elapsed * len(batches) / testing_step
            
            print(f'Testing: accuracy={test(model, train_dataset)}')
            print(f'Time since last testing (seconds) {elapsed}')
            print(f'Estimated time per epoch:')
            print(f'    seconds = {time_per_epoch}')
            print(f'    minutes = {time_per_epoch / 60}')
            print(f'    hours   = {time_per_epoch / 60 / 60}')
            print(f'----------------------------------------')


@@@@@@@@@@@@@@@@@@@@@
EPOCH: 1
@@@@@@@@@@@@@@@@@@@@@


2022-04-23 04:11:15,076	WARNING worker.py:1398 -- WARNING: 48 PYTHON worker processes have been started on node: 5c46cbfedfe6d952f4786ccede9ea3259d6b0ea75446959eb9a6b0b9 with address: 127.0.0.1. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
2022-04-23 04:11:17,147	WARNING worker.py:1398 -- WARNING: 61 PYTHON worker processes have been started on node: 5c46cbfedfe6d952f4786ccede9ea3259d6b0ea75446959eb9a6b0b9 with address: 127.0.0.1. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).


Testing: accuracy=0.1484375
Time since last testing (seconds) 20.91240096092224
Estimated time per epoch:
    seconds = 195.9491970038414
    minutes = 3.265819950064023
    hours   = 0.054430332501067054
----------------------------------------
Testing: accuracy=0.33984375
Time since last testing (seconds) 13.093400239944458
Estimated time per epoch:
    seconds = 122.68516024827957
    minutes = 2.0447526708046597
    hours   = 0.03407921118007766
----------------------------------------
Testing: accuracy=0.509765625
Time since last testing (seconds) 15.629865646362305
Estimated time per epoch:
    seconds = 146.4518411064148
    minutes = 2.4408640184402466
    hours   = 0.04068106697400411
----------------------------------------


KeyboardInterrupt: 

In [297]:
idx = random.randint(0, len(train_dataset))
inp, _, img = train_dataset[idx]

res = model.forward(inp)
pred = list(res).index(res.max())
print(pred)
img

6


In [286]:

print(train_dataset[0][0])

[-0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0

In [ ]:
import time

time.time()

In [ ]:
softmax_d(np.array(39))

In [ ]:
IND = random.randint(1, 10000)
inp = train_dataset[IND][0]
tar = train_dataset[IND][1]

res = model.forward(inp)
print(res)
print(tar)
print(sum(abs(res - tar)))